In [93]:
import torch
import cv2
import numpy as np
import math

OUTPUT_DIR = "./output/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def get_theta_torch(x, y):
    theta = torch.where(y < 0, (-1) * torch.atan2(y, x), 2 * math.pi - torch.atan2(y, x))
    return theta

def create_equirectangular_to_bottom_and_top_map(input_w, input_h, output_sqr, z):
    x, y = torch.meshgrid(torch.linspace(-output_sqr, output_sqr-1, output_sqr), 
                          torch.linspace(-output_sqr, output_sqr-1, output_sqr),indexing='ij')
    x, y = x.to(device), y.to(device)
    z = torch.tensor(z).to(device)
    
    rho = torch.sqrt(x**2 + y**2 + z**2)
    norm_theta = get_theta_torch(x, y) / (2 * math.pi)
    norm_phi = (math.pi - torch.acos(z / rho)) / math.pi
    ix = norm_theta * input_w
    iy = norm_phi * input_h

    ix = torch.remainder(ix, input_w)
    iy = torch.remainder(iy, input_h)
    
    return ix.cpu().numpy(), iy.cpu().numpy()


def create_equirectangular_to_front_and_back_map(input_w, input_h, output_sqr, x):
    z, y = torch.meshgrid(torch.linspace(-output_sqr, output_sqr-1, output_sqr), 
                          torch.linspace(-output_sqr, output_sqr-1, output_sqr),indexing='ij')
    z, y = z.to(device), y.to(device)
    x = torch.tensor(x).to(device)
    
    rho = torch.sqrt(x**2 + y**2 + z**2)
    norm_theta = get_theta_torch(x, y) / (2 * math.pi)
    norm_phi = (math.pi - torch.acos(z / rho)) / math.pi
    ix = norm_theta * input_w
    iy = norm_phi * input_h

    ix = torch.remainder(ix, input_w)
    iy = torch.remainder(iy, input_h)
    
    return ix.cpu().numpy(), iy.cpu().numpy()


def create_equirectangular_to_left_and_right_map(input_w, input_h, output_sqr, y):
    x, z = torch.meshgrid(torch.linspace(-output_sqr, output_sqr-1, output_sqr), 
                          torch.linspace(-output_sqr, output_sqr-1, output_sqr),indexing='ij')
    x, z = x.to(device), z.to(device)
    y = torch.tensor(y).to(device)
    
    rho = torch.sqrt(x**2 + y**2 + z**2)
    norm_theta = get_theta_torch(x, y) / (2 * math.pi)
    norm_phi = (math.pi - torch.acos(z / rho)) / math.pi
    ix = norm_theta * input_w
    iy = norm_phi * input_h

    ix = torch.remainder(ix, input_w)
    iy = torch.remainder(iy, input_h)
    
    return ix.cpu().numpy(), iy.cpu().numpy()



def create_cube_imgs_pad(img, margin=10):
    input_h, input_w, _ = img.shape
    output_sqr = int(input_w / 4)
    normalized_f = 1

    z = (output_sqr / normalized_f)
    bottom_map_x, bottom_map_y = create_equirectangular_to_bottom_and_top_map(input_w, input_h, output_sqr+margin*2, z)
    bottom_img = cv2.remap(img, bottom_map_x.astype("float32"), bottom_map_y.astype("float32"), cv2.INTER_CUBIC)

    z = (-1) * (output_sqr / normalized_f)
    top_map_x, top_map_y = create_equirectangular_to_bottom_and_top_map(input_w, input_h, output_sqr+margin*2, z)
    top_img = cv2.remap(img, top_map_x.astype("float32"), top_map_y.astype("float32"), cv2.INTER_CUBIC)
    top_img = cv2.flip(top_img, 0)

    x = (-1) * (output_sqr / normalized_f)
    front_map_x, front_map_y = create_equirectangular_to_front_and_back_map(input_w, input_h, output_sqr+margin*2, x)
    front_img = cv2.remap(img, front_map_x.astype("float32"), front_map_y.astype("float32"), cv2.INTER_CUBIC)

    x = output_sqr / normalized_f
    back_map_x, back_map_y = create_equirectangular_to_front_and_back_map(input_w, input_h, output_sqr+margin*2, x)
    back_img = cv2.remap(img, back_map_x.astype("float32"), back_map_y.astype("float32"), cv2.INTER_CUBIC)
    back_img = cv2.flip(back_img, 1)

    y = (-1) * (output_sqr / normalized_f)
    left_map_x, left_map_y = create_equirectangular_to_left_and_right_map(input_w, input_h, output_sqr+margin*2, y)
    left_img = cv2.remap(img, left_map_x.astype("float32"), left_map_y.astype("float32"), cv2.INTER_CUBIC)
    left_img = cv2.rotate(left_img, cv2.ROTATE_90_CLOCKWISE)

    y = output_sqr / normalized_f
    right_map_x, right_map_y = create_equirectangular_to_left_and_right_map(input_w, input_h, output_sqr+margin*2, y)
    right_img = cv2.remap(img, right_map_x.astype("float32"), right_map_y.astype("float32"), cv2.INTER_CUBIC)
    right_img = cv2.flip(right_img, 1)
    right_img = cv2.rotate(right_img, cv2.ROTATE_90_COUNTERCLOCKWISE)

    return [back_img, bottom_img, front_img, left_img, right_img, top_img], output_sqr

In [94]:
def create_cube_map(back_img, bottom_img, front_img, left_img, right_img, top_img, output_sqr):
    cube_map_img = np.zeros((3 * output_sqr, 4 * output_sqr, 3))
    cube_map_img[output_sqr:2*output_sqr, 3*output_sqr:4*output_sqr] = back_img
    cube_map_img[2*output_sqr:3*output_sqr, output_sqr:2*output_sqr] = bottom_img
    cube_map_img[output_sqr:2*output_sqr, output_sqr:2*output_sqr] = front_img
    cube_map_img[output_sqr:2*output_sqr, 0:output_sqr] = left_img
    cube_map_img[output_sqr:2*output_sqr, 2*output_sqr:3*output_sqr] = right_img
    cube_map_img[0:output_sqr, output_sqr:2*output_sqr] = top_img
    return cube_map_img

In [95]:
image_path = "../data/example/farm4/O.png"
img = cv2.imread(image_path)
print(img.shape, type(img))
[back_img, bottom_img, front_img, left_img, right_img, top_img], output_sqr = create_cube_imgs_pad(img, margin=0)
cube_map_img = create_cube_map(back_img, bottom_img, front_img, left_img, right_img, top_img, output_sqr)

cv2.imwrite(f"{OUTPUT_DIR}bottom.png", bottom_img)
cv2.imwrite(f"{OUTPUT_DIR}top.png", top_img)
cv2.imwrite(f"{OUTPUT_DIR}front.png", front_img)
cv2.imwrite(f"{OUTPUT_DIR}back.png", back_img)
cv2.imwrite(f"{OUTPUT_DIR}left.png", left_img)
cv2.imwrite(f"{OUTPUT_DIR}right.png", right_img)
cv2.imwrite(f"{OUTPUT_DIR}cube_map.png", cube_map_img)

(512, 1024, 3) <class 'numpy.ndarray'>


True

In [69]:
import torch

def cube_coord_to_3d_vector(face: str, cor_xy: (float, float), width: int) -> torch.Tensor:
    x, y = cor_xy
    x -= width
    y -= width

    
    # Depending on the face, compute the 3D direction
    if face == "front":
        return torch.tensor([width, y, -x])#torch.tensor([x, y, width/2])
    elif face == "back":
        return torch.tensor([-width, -y, -x])
    elif face == "right":
        return torch.tensor([-y, width, -x])#torch.tensor([x, width/2, y])
    elif face == "left":
        return torch.tensor([y, -width, -x])#torch.tensor([width/2, y, x])
    elif face == "top":
        return torch.tensor([x, y, width])#torch.tensor([x, y, width/2])#torch.tensor([x, y, -width/2]) # ok
    elif face == "bottom":
        return torch.tensor([-x, y, -width])#torch.tensor([x, y, -width/2])
    else:
        raise ValueError(f"Invalid face name: {face}")
    
def vector_to_equirectangular_coord(vec: torch.Tensor, width: int) -> (float, float):
    # Convert 3D cartesian coordinates to spherical coordinates
    r = torch.sqrt(torch.sum(vec**2))
    theta = torch.acos(vec[2] / r)  # polar angle (0 <= theta <= pi)
    phi = torch.atan2(vec[1], vec[0])  # azimuthal angle (-pi <= phi <= pi)
    
    # Convert spherical coordinates to 2D equirectangular coordinates
    # Taking into account the new desired width and height
    x = width * 8 * (phi + torch.pi) / (2 * torch.pi)
    y = width * 4 * theta / torch.pi
    
    return x.item(), y.item()

def cube_to_equirectangular_coord(face: str, cor_xy: (float, float), width: int) -> (float, float):
    vec = cube_coord_to_3d_vector(face, cor_xy, width)
    #print(vector_to_equirectangular_coord(vec, width))
    return vector_to_equirectangular_coord(vec, width)


if __name__ == "__main__":
    # Test the function
    test_face = "front"
    test_cor_xy = (256, 256)
    test_width = 512

    print(cube_to_equirectangular_coord(test_face, test_cor_xy, test_width))

(1745.748779296875, 749.854248046875)


In [2]:
from kornia.feature import LoFTR

In [3]:
matcher = LoFTR(pretrained='outdoor')

Downloading: "http://cmp.felk.cvut.cz/~mishkdmy/models/loftr_outdoor.ckpt" to /home/imonalc/.cache/torch/hub/checkpoints/loftr_outdoor.ckpt
100%|██████████| 44.2M/44.2M [04:58<00:00, 155kB/s] 
